# Augment Dataset for Mask RCNN
Auther : moriitkys  
LICENSE : MIT

# <font color="OrangeRed">データセットのバックアップを取ってから実行してください</font>
## Make a backup of your dataset before running this program

主にMask R-CNN用のデータセット拡張をするためのプログラムです。セルを実行すると、TkinterのUIパネルが表示され、以下を変更可能です。
- データセットの種類（RGBかRGBDかRGBDT（他の組み合わせも可）かそれらにマスク画像を付けるか）
- 拡張の種類（反転、輝度変化、ペッパーノイズ、カットアウト、変形）
拡張のパラメータとしては、輝度の明るさの度合い（0.5より小さいほど暗く、大きいほど明るい）と変形処理の回数（大きいほど爆発的に水増しされる）がある。
拡張のイメージは以下の図を参考。
<img src="Documents/imgs/Augmentation_types.PNG" width="400">
<img src="Documents/imgs/AugDataSet_structure_all.png" width="600">

In [ ]:
#Settings and prepare your dataset
import glob
import os
import sys
import csv

#numpy==1.18.4 ->  numpy-1.16.4 (管理者権限でAnaconda Prompt)
#h5py==  -> 2.8.0rc1 (pip install h5py==2.8.0rc1)
import numpy as np
import random
import matplotlib.pyplot as plt

import PIL
from PIL import Image
import cv2

from pathlib import Path
import shutil

from sklearn.model_selection import train_test_split

import mylib.augdataset as augdataset
import mylib.create_panel as create_panel
import mylib.config as config
config_param = config.ConfigParameters()

# ------ Setting panels ------
import tkinter
from tkinter import messagebox

dataset_type_all = config_param.dataset_type_all
augmentation_type_all = config_param.augmentation_type_all
key_for_searching_foldername = config_param.key_for_ads

list_selected_folders = []
flags_dataset_type = []
flags_aug_type = []

setting_panel = create_panel.CreatePanel(dataset_type_all, 
                                         augmentation_type_all,
                                         key_for_searching_foldername)
setting_panel.create_buttons()#If you push "start", exit this line.


# ----- Check the selection correctness -----
list_selected_folders = setting_panel.list_selected_folders
print("list_selected_folders", list_selected_folders)
flags_dataset_type = setting_panel.flags_dataset_type
print("flags_dataset_type", flags_dataset_type)
flags_aug_type = setting_panel.flags_aug_type
print("flags_aug_type", flags_aug_type)

list_selected_lengths = [
    len(setting_panel.list_selected_folders),
    len(setting_panel.list_selected_datatype),
    len(setting_panel.list_selected_augtype)
]

root = tkinter.Tk()
root.withdraw()
print(list_selected_lengths)
if 0 in list_selected_lengths:
    messagebox.showerror('Error エラー', 'folders, datatypes, augtypesのいずれかの選択が正しくありません。')
    sys.exit()
        
# ------ Get params ------
aug_scale_factor = float(setting_panel.var_df_number.get())  #1~100
gc_ratio = float(setting_panel.var_gc_degree.get())  #0.0=1.0

#dataset_type_names = {'rgb':'rgb', 'd':'d', 't':'t', 'mask':'_and_mask'}
#dataset_type = {'rgb':'_c_raw.png', 'mask':'_c_raw_mask.png'}#Initialize
dataset_type = {}#Initialize

for key in flags_dataset_type:
    if flags_dataset_type[key]:
        dataset_type[key] = dataset_type_all[key]
            
#target_folder = setting_panel.target_folder

# ------ Do Augmentation (main process) ------

aug_dataset = augdataset.AugDataSet(flags_aug_type = flags_aug_type,
                                        aug_scale_factor = aug_scale_factor,
                                        gc_ratio = gc_ratio,
                                        img_size = config_param.img_size,
                                        control_warning_datasize = config_param.control_warning_datasize)

for path_selected_folder in list_selected_folders:
    aug_dataset.do_augmentation(path_selected_folder = path_selected_folder, dataset_type = dataset_type)
    
print("Augmentation completed")